In [2]:
#####导入相关包#####
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

In [16]:
####函数#####
class tsp(object):
    
    def _init_(self):
        self.df=pd.read_excel('./cities.xlsx')
        
    def create_data_model(self):
        data = {}
        temp=pd.read_excel('./distance.xlsx',index_col=0)
#         print(temp)
        data['distance_matrix']=temp.values/1000
#         print(data['distance_matrix'])
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data
    
    def get_solution(self,manager, routing, solution):
        print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        index = routing.Start(0)
#         plan_output = '车辆的路径:\n'
        route=[]
        route_distance = 0
        while not routing.IsEnd(index):
#         plan_output += ' {} ->'.format(city_names[manager.IndexToNode(index)])
            index_show=manager.IndexToNode(index)
            route.append(index_show)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
#         plan_output += city_names[manager.IndexToNode(index)]
#         print(plan_output)
#         print('{}公里'.format(route_distance))
        return route,route_distance

    def work(self):
        # 初始化数据
        data = self.create_data_model()

        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # 创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 计算两点之间的距离
        def distance_callback(from_index, to_index):
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        route,route_distance=self.get_solution(manager, routing, solution)
        return route,route_distance           

In [18]:
### 运行主函数 ###
if __name__ == '__main__':
    model=tsp()
    route,route_distance=model.work()
    print(route)
    print(route_distance)

总行驶里程: 19799 公里
[0, 6, 22, 21, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 10, 9, 8, 15, 13, 28, 29, 14, 20, 17, 18, 19, 7, 5, 16, 1, 4, 2, 3]
19799
